# 1. Set-Up

In [1]:
import json
import pandas as pd

In [ ]:
# Load the JSON data (replace with your actual file paths)
with open('C:/Users/Emilia/Documents/Uni Helsinki/Year Three/AMO Freelance/raw data/national party included/9term_apas_w_nationalParty.json', 'r', encoding='utf-8') as f1, open('C:/Users/Emilia/Documents/Uni Helsinki/Year Three/AMO Freelance/assistant task/10 term/raw data/national party, 10-11-2024/mep_assistants_national_party.json', 'r', encoding='utf-8') as f2:
    term9 = json.load(f1)
    term10 = json.load(f2)

def clean_invalid_meps(json_data):
    """
    This function filters out MEP entries with missing 'group', 'national_party', or 'country'.
    It returns the cleaned list of MEPs and also tracks the MEPs with missing fields.
    """
    cleaned_json_data = []
    invalid_meps = []

    # Iterate through the JSON data
    for entry in json_data:
        name = entry.get('name')
        group = entry.get('group')
        national_party = entry.get('national_party')
        country = entry.get('country')

        # If any critical field is missing, add to invalid_meps and skip this MEP
        if not group or not national_party or not country:
            invalid_meps.append({
                'name': name,
                'group_missing': not group,
                'national_party_missing': not national_party,
                'country_missing': not country
            })
            continue  # Skip this MEP and do not add it to cleaned_json_data

        # Add to cleaned data if no fields are missing
        cleaned_json_data.append(entry)

    return cleaned_json_data, invalid_meps

term9_cleaned, invalid_meps = clean_invalid_meps(term9)


<built-in method count of list object at 0x000001F2AF23B840>


convert to data frame

In [ ]:
# Convert data to DataFrame
df_9th = pd.json_normalize(term9_cleaned)
df_9th['term'] = 9  # Add term info
df_10th = pd.json_normalize(term10)
df_10th['term'] = 10


all_meps = pd.concat([df_9th, df_10th], ignore_index=True)
print(all_meps.columns)

                  name                                              group  \
0      Abir AL-SAHLANI                                 Renew Europe Group   
1          Adam BIELAN        European Conservatives and Reformists Group   
2         Adam JARUBAS  Group of the European People's Party (Christia...   
3   Adina-Ioana VĂLEAN  Group of the European People's Party (Christia...   
4  Adrian-Dragoş BENEA  Group of the Progressive Alliance of Socialist...   

   country              national_party date_scraped  \
0   Sweden               Centerpartiet   2024-02-05   
1   Poland      Prawo i Sprawiedliwość   2019-09-22   
2   Poland  Polskie Stronnictwo Ludowe   2019-09-21   
3  Romania   Partidul Naţional Liberal   2019-10-31   
4  Romania    Partidul Social Democrat   2019-09-03   

                    assistants.Accredited assistants  \
0  [Anna Sophia BENGTSSON, John August HULTENGAAR...   
1                                                NaN   
2  [Sylwia Joanna BETKOWSKA, Borys Mar

# 2. Standardize & Transform

In [13]:
# Merge 'mep_group' and 'group'
all_meps['group'] = all_meps['mep_group'].combine_first(all_meps['group'])

# Merge 'mep_national_party' and 'national_party'
all_meps['national_party'] = all_meps['mep_national_party'].combine_first(all_meps['national_party'])

# Drop the redundant columns after merging
all_meps.drop(columns=['mep_group', 'mep_national_party'], inplace=True)

# Check the updated DataFrame
print(all_meps.tail())

                            name  \
1408           Nicola ZINGARETTI   
1409             Kosma ZŁOTOWSKI   
1410  Juan Ignacio ZOIDO ÁLVAREZ   
1411               Željana ZOVKO   
1412                  Milan ZVER   

                                                  group   country  \
1408  Group of the Progressive Alliance of Socialist...     Italy   
1409        European Conservatives and Reformists Group    Poland   
1410  Group of the European People's Party (Christia...     Spain   
1411  Group of the European People's Party (Christia...   Croatia   
1412  Group of the European People's Party (Christia...  Slovenia   

                      national_party date_scraped  \
1408             Partito Democratico          NaN   
1409          Prawo i Sprawiedliwość          NaN   
1410                 Partido Popular          NaN   
1411  Hrvatska demokratska zajednica          NaN   
1412   Slovenska demokratska stranka          NaN   

                       assistants.Accredited assi

In [14]:
# keep only columns that we will use in analysis
columns_to_keep = [
    'name',
    'national_party', 
    'group', 
    'country', 
    'term', 
    'assistants.Accredited assistants', 
    'assistants.Accredited assistants (grouping)'
]

# filter the DataFrame
meps_apas = all_meps[columns_to_keep]

# rename the columns by removing the 'assistants.' prefix
meps_apas.columns = [col.replace('assistants.', '') for col in meps_apas.columns]

print(meps_apas.columns)

Index(['name', 'national_party', 'group', 'country', 'term',
       'Accredited assistants', 'Accredited assistants (grouping)'],
      dtype='object')


In [15]:
# Create a mapping dictionary for party names to abbreviations
group_abbreviations = {
    'Renew Europe Group': 'Renew',
    'European Conservatives and Reformists Group': 'ECR',
    "Group of the European People's Party (Christian Democrats)": 'EPP',
    'Group of the Progressive Alliance of Socialists and Democrats in the European Parliament': 'S&D',
    'Identity and Democracy Group': 'ID',
    'Group of the Greens/European Free Alliance': 'G/EFA',
    'Confederal Group of the European United Left - Nordic Green Left': 'GUE/NGL',
    'Non-attached Members': 'NI',
    'The Left group in the European Parliament - GUE/NGL': 'GUE/NGL',
    'Group of the European United Left - Nordic Green Left': 'GUE/NGL',
    'Patriots for Europe Group': 'Patriots',
    'Europe of Sovereign Nations Group': 'ESN'
}

meps_apas.loc[:, 'group_abbr'] = meps_apas['group'].map(group_abbreviations)
#print(meps_apas['national_party'].unique())
print(meps_apas['group_abbr'].unique())
# #print(meps_apas.columns)

# Iterate through the rows of the DataFrame
for index, row in meps_apas.iterrows():
    if pd.isna(row['group_abbr']):
        print(row['name'])

['Renew' 'ECR' 'EPP' 'S&D' 'ID' 'G/EFA' 'GUE/NGL' 'NI' 'ESN' 'Patriots']


C:\Users\Emilia\AppData\Local\Temp\ipykernel_20720\3012400087.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meps_apas.loc[:, 'group_abbr'] = meps_apas['group'].map(group_abbreviations)


In [16]:
# Function to extract relevant assistants with type information
def extract_assistants(row):
    assistants = []
    # Define relevant groups and their corresponding types
    assistant_groups = {
        'Accredited assistants': 'accredited',
        'Accredited assistants (grouping)': 'accredited_grouping'
    }
    
    for group, assistant_type in assistant_groups.items():
        # Check if the group exists in the row and is a list
        if group in row and isinstance(row[group], list):
            for name in row[group]:
                assistants.append({
                    'assistant_name': name,
                    'mep_name': row['name'],
                    'group_full': row['group'],
                    'group_abbr': row['group_abbr'],
                    'mep_party': row['national_party'],
                    'country': row['country'],
                    'term': row['term'],
                    'assistant_type': assistant_type  # Add assistant type
                })
    return assistants

# Flatten assistants for easier comparison
assistant_data = pd.DataFrame(
    [item for sublist in meps_apas.apply(extract_assistants, axis=1) for item in sublist]
)
print(assistant_data)

                         assistant_name         mep_name  \
0                 Anna Sophia BENGTSSON  Abir AL-SAHLANI   
1               John August HULTENGAARD  Abir AL-SAHLANI   
2                  Tyra Louise LUNDBERG  Abir AL-SAHLANI   
3     Linn Christina Brunhilde OETTERLI  Abir AL-SAHLANI   
4               Sylwia Joanna BETKOWSKA     Adam JARUBAS   
...                                 ...              ...   
5852          Andréa Laure Marie MOULIN    Željana ZOVKO   
5853                         Polona KEK       Milan ZVER   
5854                     Petra SKRINJAR       Milan ZVER   
5855                        Peter SUHEL       Milan ZVER   
5856                     Dominik STRAKL       Milan ZVER   

                                             group_full group_abbr  \
0                                    Renew Europe Group      Renew   
1                                    Renew Europe Group      Renew   
2                                    Renew Europe Group      Renew   

In [17]:
# FIRST REMAP THE NAMES
from fuzzywuzzy import fuzz

def find_similar_names(df, same_term=True):
    similar_pairs = []

    # Iterate through each MEP
    for mep in df['mep_name'].unique():
        # Filter for the current MEP
        if same_term:
            mep_data = df[df['mep_name'] == mep]
        else:
            mep_data = df[df['mep_name'] == mep]  # No term filter for this case

        # Check if the DataFrame contains 'term' column
        term = mep_data['term'].iloc[0] if 'term' in mep_data.columns else "Unknown term"

        names = mep_data['assistant_name'].tolist()

        # Check for similarity between each pair of assistant names
        for i in range(len(names)):
            for j in range(i + 1, len(names)):
                # Calculate similarity score
                score = fuzz.ratio(names[i].lower(), names[j].lower())
                
                # Adjusting the threshold between 90 and 99
                if 75 <= score < 100:  # Use the desired range here
                    similar_pairs.append((names[i], names[j], mep, term))

    return similar_pairs

# Find similar names considering only the same term
#similar_names_same_term = find_similar_names(assistant_data, same_term=True)
#print("Similar assistant names for the same MEP in the same term:")
#for name1, name2, mep in similar_names_same_term:
#    print(f"{name1} and {name2} for {mep} are similar.")

# create remapping based on similar names
name_mapping = {
    "Magdalena NOWACKA": "Magdalena HILLS-NOWACKA", 
    "PAULA SENDIN RODRIGUEZ": "Paula SENDÍN RODRIGUEZ",
    "Eleonora Nikolaycheva GUIGOVA": "Eleonora Nikolaycheva GUIGOVA-NOSKER",
    "Eleonora Nikolaycheva GUIGOVA-NOSKER": "Eleonora Nikolaycheva GUIGOVA-NOSKER",
    "Stefanie SIFFT": "Stefanie SIFFT",
    "Stefanie Gabi SIFFT": "Stefanie SIFFT",
    "Anne-Cecile Juliette GAULT": "Anne-Cecile Juliette GAULT",
    "Anne-Cecile Juliette Rachel GAULT": "Anne-Cecile Juliette GAULT",
    "Sophie Anne Geraldine Marie GUIL": "Sophie Anne Geraldine Marie GUIL",
    "Sophie Anne Geraldine Marie Genevieve GUIL": "Sophie Anne Geraldine Marie GUIL",
    "Ana LOPEZ GONZALEZ": "ANA LÓPEZ GONZÁLEZ",
    "ANA LÓPEZ GONZÁLEZ": "ANA LÓPEZ GONZÁLEZ",
    "Claudia MARTINEZ MUNOZ": "Claudia MARTÍNEZ MUÑOZ",
    "CLAUDIA MARTÍNEZ MUÑOZ": "Claudia MARTÍNEZ MUÑOZ",
    "Maria Mercedes GARCIA MUNOZ": "MARIA MERCEDES GARCIA MUÑOZ",
    "MARIA MERCEDES GARCIA MUÑOZ": "MARIA MERCEDES GARCIA MUÑOZ",
    "Gilles Willy B SEGERS": "GILLES WILLY SEGERS",
    "GILLES WILLY SEGERS": "GILLES WILLY SEGERS",
    "Magdalena GONZALEZ GOZALBO": "Maria Magdalena GONZALEZ GOZALBO",
    "Maria Magdalena GONZALEZ GOZALBO": "Maria Magdalena GONZALEZ GOZALBO",
    "Arturo VILLARROYA GONZALEZ": "Arturo VILLARROYA GONZÁLEZ",
    "Fernando Jose NUNEZ ROBRES PATINO": "Fernando Jose NUNEZ-ROBRES PATINO",
    "Fernando Jose NUNEZ-ROBRES PATINO": "Fernando Jose NUNEZ-ROBRES PATINO",
    "Bibiana CARRETO PEREZ BARBADILLO": "BIBIANA CARRETO PÉREZ BARBADILLO",
    "BIBIANA CARRETO PÉREZ BARBADILLO": "BIBIANA CARRETO PÉREZ BARBADILLO",
    "Magdalena GONZALEZ GOZALBO": "Maria Magdalena GONZALEZ GOZALBO",
    "PAULA SENDIN RODRIGUEZ": "Paula SENDÍN RODRIGUEZ",
    "Paula SENDIN RODRIGUEZ": "Paula SENDÍN RODRIGUEZ",
    "MARIA MERCEDES GARCIA MUNOZ": "MARIA MERCEDES GARCIA MUÑOZ", 
    "ARTURO VILLARROYA GONZALEZ": "Arturo VILLARROYA GONZÁLEZ",
    "ARTURO VILLARROYA GONZALEZ":"Arturo VILLARROYA GONZÁLEZ", 
    "Marlene Elisabeth SEELMAIER": "Marlene Elisabeth STROEHM",          # NAMED CHANGED, CURRENT REMAPPING FOUND IN 10th term 
    "Olivia Jeanna Beatrice ANDERSSON": "Olivia Jeanna Beatrice STÅNGE", # NAMED CHANGED, CURRENT REMAPPING FOUND IN 10th term
    "Sirle ROSENFELDT": "Sirle ROSENFELDT - KOOP",                       # NAMED CHANGED, CURRENT REMAPPING FOUND IN 10th term
    "Saara ANTTINEN": "Saara ANTTINEN-KHUMALO",                          # NAMED CHANGED, CURRENT REMAPPING FOUND IN 10th term
    "Nikolina BRKOVIC": "Nikolina BRKOVIC STANCHEVA", 
    "Josep/Pepe MERCADAL BAQUERO": "Pepe MERCADAL BAQUERO",               # CHANGED HIS NAME TO PEPE
    "Josep MERCADAL BAQUERO": "Pepe MERCADAL BAQUERO" 
}

# remap 
assistant_data['assistant_name'] = assistant_data['assistant_name'].replace(name_mapping)

# check that name remapping worked
# Find similar names considering only the same term
similar_names_same_term = find_similar_names(assistant_data, same_term=True)
print("Similar assistant names for the same MEP in the same term:")
for name1, name2, mep, term in similar_names_same_term:
    print(f"{name1} and {name2} for {mep} in term {term} are similar.")

Similar assistant names for the same MEP in the same term:
Silvia RIGHI and SILVIA DRAGONI for Elisabetta GUALMINI in term 9 are similar.
Silvia RIGHI and SILVIA DRAGONI for Elisabetta GUALMINI in term 9 are similar.
Dirk FRIEDRICH and Pierre FRIEDRICH for Jörg MEUTHEN in term 9 are similar.
Jakub PUSZKARSKI and Jakub KARWANSKI for Magdalena ADAMOWICZ in term 9 are similar.
Michal MOJTO and Michaela MOJTOVÁ for Monika BEŇOVÁ in term 9 are similar.
CARLOS CAMPILLO PEREZ and IGNACIO CAMPILLO PEREZ for Mireia BORRÁS PABÓN in term 10 are similar.
CARLOS CAMPILLO PEREZ and IGNACIO CAMPILLO PEREZ for Jorge BUXADÉ VILLALBA in term 10 are similar.
CARLOS CAMPILLO PEREZ and IGNACIO CAMPILLO PEREZ for Juan Carlos GIRAUTA VIDAL in term 10 are similar.
Michal MOJTO and Michaela MOJTOVÁ for Erik KALIŇÁK in term 10 are similar.
CARLOS CAMPILLO PEREZ and IGNACIO CAMPILLO PEREZ for Jorge MARTÍN FRÍAS in term 10 are similar.
CARLOS CAMPILLO PEREZ and IGNACIO CAMPILLO PEREZ for Margarita DE LA PISA CARR

In [18]:
# Function to find exact matches
def find_exact_matches(df):
    exact_matches = {}

    # Group by 'mep_name', 'term', and 'assistant_type'
    for (mep, term, assistant_type), group in df.groupby(['mep_name', 'term', 'assistant_type']):
        # Normalize names by stripping whitespace and converting to lowercase
        normalized_names = group['assistant_name'].str.strip().str.lower()
        
        # Count occurrences of each normalized name
        duplicates = normalized_names.value_counts()
        
        # Only keep names that occur more than once
        duplicate_names = duplicates[duplicates > 1].index.tolist()
        
        if duplicate_names:
            # Store matches grouped by MEP, term, and assistant_type
            exact_matches[(mep, term, assistant_type)] = duplicate_names

    return exact_matches

# Get exact matches
exact_matches = find_exact_matches(assistant_data)

# Print out exact matches
print("Exact matches in MEP categories (for the same term and assistant_type):")
for (mep, term, assistant_type), names in exact_matches.items():
    print(f"\nMEP: {mep} | Term: {term} | Assistant Type: {assistant_type}")
    for name in names:
        print(f" - {name}")

Exact matches in MEP categories (for the same term and assistant_type):


In [19]:
# Create a mask to drop rows with exact matches
def drop_exact_matches(df, exact_matches):
    # Initialize a mask with all True values
    mask = pd.Series(True, index=df.index)

    # Iterate over the exact matches to identify rows to drop
    for (mep, term, assistant_type), names in exact_matches.items():
        for name in names:
            # Normalize the assistant_name for comparison
            condition = (
                (df['mep_name'] == mep) &
                (df['term'] == term) &
                (df['assistant_type'] == assistant_type) &
                (df['assistant_name'].str.strip().str.lower() == name)
            )
            # Update the mask to exclude matching rows
            mask &= ~condition

    # Return the filtered dataframe
    return df[mask]

# Drop exact matches from the dat
filtered_apas = drop_exact_matches(assistant_data, exact_matches)

# Print the filtered dataset
print(filtered_apas)

# save unique mep names to an excel
#assistant_data.to_excel('assistants_ALL_CORRECT.xlsx', index=False, engine='openpyxl')

                         assistant_name         mep_name  \
0                 Anna Sophia BENGTSSON  Abir AL-SAHLANI   
1               John August HULTENGAARD  Abir AL-SAHLANI   
2                  Tyra Louise LUNDBERG  Abir AL-SAHLANI   
3     Linn Christina Brunhilde OETTERLI  Abir AL-SAHLANI   
4               Sylwia Joanna BETKOWSKA     Adam JARUBAS   
...                                 ...              ...   
5852          Andréa Laure Marie MOULIN    Željana ZOVKO   
5853                         Polona KEK       Milan ZVER   
5854                     Petra SKRINJAR       Milan ZVER   
5855                        Peter SUHEL       Milan ZVER   
5856                     Dominik STRAKL       Milan ZVER   

                                             group_full group_abbr  \
0                                    Renew Europe Group      Renew   
1                                    Renew Europe Group      Renew   
2                                    Renew Europe Group      Renew   